US Energy Information Administration API calls

Detail on EIA APIv2: https://www.eia.gov/opendata/documentation.php

In [2]:
# Import packages
import requests
import json
import pandas as pd
import os

In [ ]:
#Finding a path to plant-level data
#Begin with parent group "electricity"
eia_electricity_url = "https://api.eia.gov/v2/electricity/?api_key=iPjXaaJSiyfWNALV62yg1Onb0VTuUgVHa0eVHgbx"
eia_electricity_response = requests.get(eia_electricity_url)
eia_electricity_json = json.loads(eia_electricity_response.text)
print(json.dumps(eia_electricity_json, separators=(",",":"), indent=4))

Output from previous cell:
{
    "response":{
        "id":"electricity",
        "name":"Electricity",
        "description":"EIA electricity survey data",
        "routes":[
            {
                "id":"retail-sales",
                "name":"Electricity Sales to Ultimate Customers",
                "description":"Electricity sales to ultimate customer by state and sector (number of customers, average price, revenue, and megawatthours of sales).  \n    Sources: Forms EIA-826, EIA-861, EIA-861M"
            },
            {
                "id":"electric-power-operational-data",
                "name":"Electric Power Operations (Annual and Monthly)",
                "description":"Monthly and annual electric power operations by state, sector, and energy source.\n    Source: Form EIA-923"
            },
            {
                "id":"rto",
                "name":"Electric Power Operations (Daily and Hourly)",
                "description":"Hourly and daily electric power operations by balancing authority.  \n    Source: Form EIA-930"
            },
            {
                "id":"state-electricity-profiles",
                "name":"State Specific Data",
                "description":"State Specific Data"
            },
            {
                "id":"operating-generator-capacity",
                "name":"Inventory of Operable Generators",
                "description":"Inventory of operable generators in the U.S.\n    Source: Forms EIA-860, EIA-860M"
            },
            {
                "id":"facility-fuel",
                "name":"Electric Power Operations for Individual Power Plants (Annual and Monthly)",
                "description":"Annual and monthly electric power operations for inidividual power plants, by energy source and prime mover\n    Source: Form EIA-923"
            }
        ]
    },
    "request":{
        "command":"/v2/electricity/",
        "params":{
            "api_key":"iPjXaaJSiyfWNALV62yg1Onb0VTuUgVHa0eVHgbx"
        }
    },
    "apiVersion":"2.0.2"
}

id : facility-fuel looks like it could have what we need

In [ ]:
#update url to view facility-fuel child cat
eia_facility_url = "https://api.eia.gov/v2/electricity/facility-fuel/?api_key=iPjXaaJSiyfWNALV62yg1Onb0VTuUgVHa0eVHgbx"
eia_facility_response = requests.get(eia_facility_url)
eia_facility_json = json.loads(eia_facility_response.text)
print(json.dumps(eia_facility_json, separators=(",",":"), indent=4))

Output from previous cell:
{
    "response":{
        "id":"facility-fuel",
        "name":"Electric Power Operations for Individual Power Plants (Annual and Monthly)",
        "description":"Annual and monthly electric power operations for inidividual power plants, by energy source and prime mover\n    Source: Form EIA-923",
        "frequency":[
            {
                "id":"monthly",
                "description":"One data point for each month.",
                "query":"M",
                "format":"YYYY-MM"
            },
            {
                "id":"quarterly",
                "description":"One data point every 3 months.",
                "query":"Q",
                "format":"YYYY-\"Q\"Q"
            },
            {
                "id":"annual",
                "description":"One data point for each calendar year.",
                "query":"A",
                "format":"YYYY"
            }
        ],
        "facets":[
            {
                "id":"plantCode",
                "description":"Plant ID and Name"
            },
            {
                "id":"fuel2002",
                "description":"Energy Source"
            },
            {
                "id":"state",
                "description":"State"
            },
            {
                "id":"primeMover",
                "description":"Prime Mover"
            }
        ],
        "data":{
            "generation":{
                "alias":"Net Generation",
                "units":"megawatthours"
            },
            "gross-generation":{
                "alias":"Gross Generation",
                "units":"megawatthours"
            },
            "total-consumption":{
                "alias":"Consumption of Fuels for Electricity Generation and Useful Thermal Output (Physical Units)"
            },
            "total-consumption-btu":{
                "alias":"Consumption of Fuels for Electricity Generation and Useful Thermal Output (BTUs)",
                "units":"MMBtu"
            },
            "consumption-for-eg":{
                "alias":"Consumption of Fuels for Electricity Generation (Physical Units)"
            },
            "consumption-for-eg-btu":{
                "alias":"Consumption of Fuels for Electricity Generation (BTUs)",
                "units":"MMBtu"
            },
            "average-heat-content":{
                "alias":"Average Heat Content of Consumed Fuels"
            }
        },
        "startPeriod":"2001-01",
        "endPeriod":"2021-07",
        "defaultDateFormat":"YYYY-MM",
        "defaultFrequency":"monthly"
    },
    "request":{
        "command":"/v2/electricity/facility-fuel/",
        "params":{
            "api_key":"iPjXaaJSiyfWNALV62yg1Onb0VTuUgVHa0eVHgbx"
        }
    },
    "apiVersion":"2.0.2"
}

Parameters needed to pull data relevant to this project:
    frequency=monthly (monthly data)
    start=2013-01
    end=2020-12
    data[]=generation
    facets[plantCode][]=62178 (testing a specific plant: Maverick Solar, CA)
          [fuel2002][]=SUN
          [primeMover][]=PV
          [state][]=AZ,CA,CO,ID,MT,NM,NV,OR,TX,UT,WA, WY

In [ ]:
#retrieve monthly, plant level PV generation data
plant_gen_url = "https://api.eia.gov/v2/electricity/facility-fuel/data/?api_key=iPjXaaJSiyfWNALV62yg1Onb0VTuUgVHa0eVHgbx&frequency=monthly&start=2013-01&end=2020-12&data[]=generation&facets[plantCode][]=62178&facets[fuel2002][]=SUN&facets[primeMover][]=PV"
plant_gen_response = requests.get(plant_gen_url)
plant_gen_json = json.loads(plant_gen_response.text)
print(json.dumps(plant_gen_json, separators=(",",":"), indent=4))

In [19]:
#retrieve monthly, plant level characteristic data
plant_char_url = "https://api.eia.gov/v2/electricity/operating-generator-capacity/data?api_key=iPjXaaJSiyfWNALV62yg1Onb0VTuUgVHa0eVHgbx&length=1&data[]=county&data[]=latitude&data[]=longitude&data[]=nameplate-capacity-mw&data[]=operating-year-month&facets[plantid][]=64750"
plant_char_response = requests.get(plant_char_url)
plant_char_json = json.loads(plant_char_response.text)
print(json.dumps(plant_char_json, separators=(",",":"), indent=4))

{
    "response":{
        "query execution":0.000100632,
        "count query execution":0.198864417,
        "total":5,
        "dateFormat":"YYYY-MM",
        "frequency":"monthly",
        "data":[
            {
                "period":"2021-11",
                "stateid":"OK",
                "stateName":"Oklahoma",
                "sector":"ipp-non-chp",
                "sectorName":"IPP Non-CHP",
                "entityid":17650,
                "entityName":"Southern Power Co",
                "plantid":64750,
                "plantName":"Glass Sands Wind Facility",
                "generatorid":1,
                "unit":null,
                "technology":"Onshore Wind Turbine",
                "energy_source_code":"WND",
                "energy-source-desc":"Wind",
                "prime_mover_code":"WT",
                "balancing_authority_code":"SWPP",
                "balancing-authority-name":"Southwest Power Pool",
                "status":"OP",
                "statusD